# Visier Aggregate Query API Tutorial

This notebook teaches you how to use the Visier Aggregate Query API. You'll learn:
- How the Visier API works
- How to structure query payloads
- Understanding dimensions, metrics, and filters
- How to modify queries for your needs
- Best practices and common patterns

We'll use the Organization Hierarchy example to demonstrate these concepts.

## Understanding the Visier Aggregate Query API

The Visier Aggregate Query API allows you to retrieve aggregated metrics (like employee count, headcount, turnover) grouped by dimensions (like Organization, Country, Function) over time periods.

### Key Concepts:
- **Metric**: What you're measuring (e.g., `employeeCount`, `headcount`)
- **Axes (Dimensions)**: How to group/break down the data
- **Filters**: Optional constraints to restrict the data
- **Time Intervals**: The time period for the query
- **Options**: How results are formatted

### API Endpoint:
```
POST /v1/data/query/aggregate
```

**Official Documentation**: This repository includes the complete OpenAPI specification (`openapi.json`) and an API reference guide (`API_REFERENCE.md`) based on the official Visier API documentation.

Let's start by setting up our environment.

In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd

# Add parent directory to path to import client
sys.path.insert(0, str(Path().resolve().parent))

from client import (
    load_query_payload_from_json,
    execute_vanilla_aggregate_query,
    convert_vanilla_response_to_dataframe,
    get_api_config
)

print("✓ Imports successful")
print("\nThe client module provides functions to:")
print("  - load_query_payload_from_json() - Load query from JSON file")
print("  - execute_vanilla_aggregate_query() - Execute query against Visier API")
print("  - convert_vanilla_response_to_dataframe() - Convert API response to DataFrame")

: 

## Understanding Query Payload Structure

A Visier aggregate query is defined as a JSON payload. Let's examine the structure of a real query payload.

In [ ]:
# Load the org hierarchy payload
payload_path = Path("examples/query_payload_examples_org_hierarchy.json")
payload = load_query_payload_from_json(str(payload_path))

print("✓ Payload loaded successfully")
print("\n" + "="*70)
print("PAYLOAD STRUCTURE BREAKDOWN")
print("="*70)

# Show the full payload structure
print("\n1. TOP LEVEL STRUCTURE:")
print("   payload")
print("   └── query")
print("       ├── source (the metric)")
print("       ├── axes (dimensions to group by)")
print("       ├── filters (optional constraints)")
print("       └── timeIntervals (time period)")
print("   └── options (result formatting)")

print("\n2. THE METRIC (what we're measuring):")
metric = payload['query']['source']['metric']
print(f"   query.source.metric = '{metric}'")
print(f"   → This tells Visier to return {metric} data")

print("\n3. THE AXES (how to group the data):")
print(f"   query.axes = [{len(payload['query']['axes'])} dimension(s)]")
for i, axis in enumerate(payload['query']['axes'], 1):
    dim_name = axis['dimensionLevelSelection']['dimension']['name']
    qualifying_path = axis['dimensionLevelSelection']['dimension']['qualifyingPath']
    level_ids = axis['dimensionLevelSelection']['levelIds']
    print(f"\n   Axis {i}: {dim_name}")
    print(f"   ├── dimension.name = '{dim_name}'")
    print(f"   ├── dimension.qualifyingPath = '{qualifying_path}'")
    print(f"   └── levelIds = {level_ids}")
    print(f"   → Groups data by {dim_name} at level(s) {', '.join(level_ids)}")

print("\n4. THE FILTERS (optional constraints):")
if 'filters' in payload['query'] and payload['query']['filters']:
    print(f"   query.filters = [{len(payload['query']['filters'])} filter(s)]")
    for i, filter_item in enumerate(payload['query']['filters'], 1):
        if 'memberSet' in filter_item:
            dim_name = filter_item['memberSet']['dimension']['name']
            values = filter_item['memberSet']['values']['included']
            print(f"\n   Filter {i}: {dim_name}")
            print(f"   └── Restricts to: {[v['path'] for v in values]}")
else:
    print("   query.filters = [] (no filters - returns all data)")

print("\n5. TIME INTERVALS (time period):")
ti = payload['query']['timeIntervals']
print(f"   query.timeIntervals.fromDateTime = '{ti['fromDateTime']}'")
print(f"   query.timeIntervals.intervalPeriodType = '{ti['intervalPeriodType']}'")
print(f"   query.timeIntervals.intervalCount = {ti['intervalCount']}")
print(f"   query.timeIntervals.direction = '{ti['direction']}'")
print(f"   → Returns {ti['intervalCount']} {ti['intervalPeriodType'].lower()}(s) going {ti['direction'].lower()} from {ti['fromDateTime']}")

## Understanding Dimensions and Level IDs

**Critical Concept**: For parent-child hierarchies like `Organization_Hierarchy`, you MUST specify actual level IDs that exist in your tenant.

### Regular Dimensions vs Parent-Child Hierarchies

**Regular Dimensions** (like `Country_Cost`, `Function`):
- Can often use the dimension name itself as the level ID
- Example: `levelIds: ["Country"]` for `Country_Cost`

**Parent-Child Hierarchies** (like `Organization_Hierarchy`):
- MUST use tenant-specific level IDs
- Common patterns: `["Level_1"]`, `["Level_2"]`, `["Level_3"]`
- Or named levels: `["Profit_Center"]`, `["Business_Unit"]`
- **You cannot use the dimension name itself**

### How to Discover Level IDs

1. Use the Dimensions API: `GET /v1/data/model/dimensions/{dimension}/levels`
2. Check your tenant's data model documentation
3. Try common patterns and see what works
4. Contact your Visier administrator

Let's see what level IDs are used in our example:

In [ ]:
# Examine the level IDs used in this query
print("LEVEL IDS IN THIS QUERY:")
print("="*70)

for i, axis in enumerate(payload['query']['axes'], 1):
    dim_name = axis['dimensionLevelSelection']['dimension']['name']
    level_ids = axis['dimensionLevelSelection']['levelIds']
    
    print(f"\n{i}. {dim_name}:")
    print(f"   Level IDs: {level_ids}")
    
    if dim_name == "Organization_Hierarchy":
        print(f"   ⚠️  This is a PARENT-CHILD hierarchy")
        print(f"   → Level IDs are tenant-specific")
        print(f"   → Common options: ['Level_1'], ['Level_2'], ['Level_3']")
        print(f"   → Or named: ['Profit_Center'], ['Business_Unit']")
    else:
        print(f"   ✓ This is a regular dimension")
        print(f"   → Level IDs are usually the dimension name or standard levels")

print("\n" + "="*70)
print("KEY TAKEAWAY:")
print("For Organization_Hierarchy, you MUST discover and use the correct")
print("level IDs for your tenant. The dimension name itself won't work!")

## Executing the Query

Now let's execute the query against the Visier API. The API will:
1. Authenticate using your credentials (from `.env` file)
2. Validate the query payload
3. Execute the query against your tenant's data
4. Return the results

**API Request Flow:**
```
Your Code → client.py → Visier API → Response → DataFrame
```

In [ ]:
# Check API configuration (credentials)
print("API CONFIGURATION:")
print("="*70)
try:
    config = get_api_config()
    print(f"✓ Host: {config.get('host', 'Not set')}")
    print(f"✓ API Key: {'Set' if config.get('apikey') else 'Not set'}")
    print(f"✓ Vanity: {config.get('vanity', 'Not set')}")
    print(f"✓ Username: {config.get('username', 'Not set')}")
    print("\n→ Credentials are loaded from .env file")
    print("→ If missing, run: python query.py --setup")
except Exception as e:
    print(f"✗ Error loading credentials: {e}")
    print("→ Run: python query.py --setup")

print("\n" + "="*70)
print("EXECUTING QUERY:")
print("="*70)
print("\nThe execute_vanilla_aggregate_query() function:")
print("  1. Builds the API request URL")
print("  2. Adds authentication headers")
print("  3. Sends POST request to /v1/data/query/aggregate")
print("  4. Returns the JSON response")

# Execute the query
print("\n→ Executing query...")
response = execute_vanilla_aggregate_query(payload=payload)

print(f"\n✓ Query executed successfully!")
print(f"\nAPI RESPONSE STRUCTURE:")
print(f"  - cells: {len(response.get('cells', []))} data cells")
print(f"  - axes: {len(response.get('axes', []))} dimension axes")
print(f"  - Each cell contains a value for a unique combination of dimensions")

## Understanding the API Response

The Visier API returns data in a structured format. Let's examine the response structure:

In [ ]:
# Examine the raw API response structure
print("API RESPONSE STRUCTURE:")
print("="*70)
print("\n1. Response Keys:")
for key in response.keys():
    print(f"   - {key}")

print("\n2. Cells (the actual data):")
if 'cells' in response and response['cells']:
    print(f"   - Total cells: {len(response['cells'])}")
    print(f"   - Each cell represents one data point")
    print(f"   - Example cell structure:")
    example_cell = response['cells'][0]
    print(f"     {json.dumps(example_cell, indent=6)[:200]}...")

print("\n3. Axes (dimension information):")
if 'axes' in response:
    print(f"   - Number of axes: {len(response['axes'])}")
    for i, axis in enumerate(response['axes'], 1):
        print(f"   - Axis {i}: {axis.get('name', 'Unknown')}")

print("\n4. Converting to DataFrame:")
print("   The convert_vanilla_response_to_dataframe() function:")
print("   - Extracts dimension values from each cell")
print("   - Extracts the metric value")
print("   - Creates a pandas DataFrame with columns for each dimension")
print("   - Makes the data easy to work with")

In [ ]:
# Convert to DataFrame
metric_id = payload['query']['source']['metric']
df = convert_vanilla_response_to_dataframe(response, metric_id=metric_id)

print("CONVERTED TO DATAFRAME:")
print("="*70)
print(f"\n✓ Rows: {len(df)}")
print(f"✓ Columns: {len(df.columns)}")
print(f"\nColumn Structure:")
print(f"  - Dimensions: {[c for c in df.columns if c not in ['Measures', 'DateInRange', 'value', 'support']]}")
print(f"  - Measures: {[c for c in df.columns if c == 'Measures']}")
print(f"  - DateInRange: Time period for each row")
print(f"  - value: The metric value (employeeCount)")
print(f"  - support: Support count (number of records contributing to this value)")

print("\n" + "="*70)
print("UNDERSTANDING THE DATA:")
print("="*70)
print("\nEach row represents:")
print("  - A unique combination of dimension values")
print("  - At a specific time period")
print("  - With the metric value for that combination")

print("\nExample rows:")
df.head(10)

## Modifying Queries: Changing the Metric

One of the most common modifications is changing what metric you're querying. Let's see how:

In [ ]:
# Example: How to change the metric
print("CHANGING THE METRIC:")
print("="*70)
print("\nCurrent metric:")
print(f"  payload['query']['source']['metric'] = '{payload['query']['source']['metric']}'")

print("\nTo change the metric, modify this field:")
print("  payload['query']['source']['metric'] = 'headcount'")
print("  payload['query']['source']['metric'] = 'turnover'")
print("  payload['query']['source']['metric'] = 'terminationCount'")

print("\n⚠️  Important:")
print("  - Metric names are tenant-specific")
print("  - Contact your Visier administrator for available metrics")
print("  - Use the Data Model API to discover metrics")
print("  - Common metrics: employeeCount, headcount, turnover, terminationCount")

print("\nExample modification:")
modified_payload = payload.copy()
# Uncomment to try a different metric:
# modified_payload['query']['source']['metric'] = 'headcount'
print("  # modified_payload['query']['source']['metric'] = 'headcount'")
print("  # Then re-execute: response = execute_vanilla_aggregate_query(payload=modified_payload)")

In [ ]:
## Modifying Queries: Adding/Removing Dimensions

You can modify which dimensions (axes) are included in your query.

In [ ]:
## Modifying Queries: Adding Filters

Filters allow you to restrict the data to specific subsets.

## Modifying Queries: Changing Time Periods

Time intervals control what time period you're querying.

In [ ]:
print("UNDERSTANDING TIME INTERVALS:")
print("="*70)

ti = payload['query']['timeIntervals']
print("\nCurrent time interval configuration:")
print(f"  fromDateTime: {ti['fromDateTime']}")
print(f"  intervalPeriodType: {ti['intervalPeriodType']}")
print(f"  intervalCount: {ti['intervalCount']}")
print(f"  direction: {ti['direction']}")

print("\nThis means:")
print(f"  → Start from: {ti['fromDateTime']}")
print(f"  → Go {ti['direction'].lower()}: {ti['intervalCount']} {ti['intervalPeriodType'].lower()}(s)")
print(f"  → Result: {ti['intervalCount']} time periods")

print("\nCommon Patterns:")

print("\n1. Year-end snapshots (2021-2025):")
print("   fromDateTime: '2026-01-01'")
print("   intervalPeriodType: 'YEAR'")
print("   intervalCount: 5")
print("   direction: 'BACKWARD'")
print("   → Returns: 2025, 2024, 2023, 2022, 2021 year-end data")

print("\n2. Point-in-time (single date):")
print("   fromDateTime: '2025-01-01'")
print("   intervalPeriodType: 'DAY'")
print("   intervalCount: 1")
print("   direction: 'BACKWARD'")
print("   → Returns: Data for 2025-01-01 only")

print("\n3. Last 12 months:")
print("   fromDateTime: '2025-12-01'")
print("   intervalPeriodType: 'MONTH'")
print("   intervalCount: 12")
print("   direction: 'BACKWARD'")
print("   → Returns: 12 monthly snapshots")

print("\n⚠️  Important:")
print("  - fromDateTime format: 'YYYY-MM-DD'")
print("  - For year-end snapshots, use start of NEXT year with BACKWARD")
print("  - intervalPeriodType options: YEAR, MONTH, DAY, etc.")
print("  - direction: BACKWARD (into past) or FORWARD (into future)")

## Understanding Query Options

Options control how results are formatted and displayed.

a

In [ ]:
# Organization by Location
if 'Organization_Hierarchy' in df.columns and 'Location' in df.columns:
    org_location = df.groupby(['Organization_Hierarchy', 'Location'])['value'].sum().unstack(fill_value=0)
    print("Employee Count: Organization × Location")
    print(org_location)
    
    # Heatmap visualization
    plt.figure(figsize=(12, 8))
    sns.heatmap(org_location, annot=True, fmt='.0f', cmap='YlOrRd', cbar_kws={'label': 'Employee Count'})
    plt.title('Employee Count: Organization × Location')
    plt.xlabel('Location')
    plt.ylabel('Organization')
    plt.tight_layout()
    plt.show()

In [ ]:
# Organization by Country
if 'Organization_Hierarchy' in df.columns and 'Country_Cost' in df.columns:
    org_country = df.groupby(['Organization_Hierarchy', 'Country_Cost'])['value'].sum().unstack(fill_value=0)
    print("Employee Count: Organization × Country")
    print(org_country)
    
    # Visualize top countries
    top_countries = org_country.sum().nlargest(10).index
    org_country_top = org_country[top_countries]
    
    plt.figure(figsize=(14, 8))
    org_country_top.plot(kind='bar', stacked=False)
    plt.title('Employee Count: Organization × Top 10 Countries')
    plt.xlabel('Organization')
    plt.ylabel('Employee Count')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

## Export Results

Save the results to CSV for further analysis or sharing..ipn

In [ ]:
# Save to CSV
output_path = Path("output/org_hierarchy_analysis.csv")
output_path.parent.mkdir(exist_ok=True)
df.to_csv(output_path, index=False)
print(f"✓ Results saved to: {output_path}")

## Customize Your Query

You can modify the payload to change:
- **Metric**: Change `query.source.metric` (e.g., `"headcount"`, `"turnover"`)
- **Dimensions**: Add/remove/modify items in `query.axes`
- **Filters**: Modify `query.filters` to restrict data
- **Time Period**: Change `query.timeIntervals`

After modifying, re-run the cells above to see the new results.

## Summary

You've learned:
- ✅ How the Visier Aggregate Query API works
- ✅ Query payload structure (source, axes, filters, timeIntervals, options)
- ✅ The difference between regular dimensions and parent-child hierarchies
- ✅ How to modify queries (metrics, dimensions, filters, time periods)
- ✅ Best practices and common pitfalls

**Remember**: Always validate your payloads with `--validate-only` before executing, and work with your Visier administrator to discover available metrics, dimensions, and level IDs for your tenant.

In [ ]:
# Example: Modify the payload
# Uncomment and modify as needed:

# # Change metric
# payload['query']['source']['metric'] = 'headcount'

# # Add a filter
# payload['query']['filters'] = [{
#     "memberSet": {
#         "dimension": {"name": "Employment_Type", "qualifyingPath": "Employee"},
#         "values": {"included": [{"path": ["Full-time shifts"]}]}
#     }
# }]

# # Change time period
# payload['query']['timeIntervals'] = {
#     "fromDateTime": "2025-01-01",
#     "intervalPeriodType": "YEAR",
#     "intervalCount": 3,
#     "direction": "BACKWARD"
# }

print("Modify the payload above and re-run the query cells to see changes")

focus o